In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import json
!pip install vaderSentiment
!pip install pytextrank
from nltk.corpus import wordnet 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from nltk.corpus import opinion_lexicon
import spacy
import en_core_web_sm
import pytextrank
from datetime import datetime
from datetime import timedelta
from django.db import connection

nltk.download('wordnet')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)

     |████████████████████████████████| 133kB 3.4MB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
rev_url = 'https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/reviews-6-15-20.csv'

rev = pd.read_csv(rev_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
rev.head()

,Unnamed: 0,review_id,business_id,reviews_stars,date,text
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,2,uBWpaM4_PQ1NMB3N9nYkJA,j9bWpCRwpDVfwVT_V85qeA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,3,EIcE_Xw9G5bDAY-XQ0Vosg,j9bWpCRwpDVfwVT_V85qeA,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,4,BXTxF_ijBmm5RPYELr-5RQ,j9bWpCRwpDVfwVT_V85qeA,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [5]:
rev["date"] = pd.to_datetime(rev["date"], infer_datetime_format = True)
rev.head()

,Unnamed: 0,review_id,business_id,reviews_stars,date,text
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,2,uBWpaM4_PQ1NMB3N9nYkJA,j9bWpCRwpDVfwVT_V85qeA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,3,EIcE_Xw9G5bDAY-XQ0Vosg,j9bWpCRwpDVfwVT_V85qeA,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,4,BXTxF_ijBmm5RPYELr-5RQ,j9bWpCRwpDVfwVT_V85qeA,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [6]:
import nltk
nltk.download('opinion_lexicon')
from nltk.tokenize import treebank

[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.


In [8]:
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())
tokenizer = treebank.TreebankWordTokenizer()

In [9]:
def positive(sentence):
  pos = []
  words = [word.lower() for word in tokenizer.tokenize(sentence)]
  for word in words:
    if word in pos_list:
      pos.append(word)
  return pos

def negative(sentence):
  neg = []
  words = [word.lower() for word in tokenizer.tokenize(sentence)]
  for word in words:
    if word in neg_list:
      neg.append(word)
  return neg

In [10]:
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1911030000

In [12]:
def customtokensize(text):
    return re.findall("[\w']+", str(text))

rev['tokenized_text'] = rev['text'].apply(customtokensize)
rev.head(2)

,Unnamed: 0,review_id,business_id,reviews_stars,date,text,tokenized_text
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]"
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn't, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, space, ...]"


In [13]:
stopwords = ['ve',"n't",'check-in','=','= =','u','want', 'u want', 'cuz','him',"i've",'on', 'her','told','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they', 'the', 'this','its', 'l','-','they','this',"don't",'the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',',']

def filter_stopwords(text):
  nonstopwords = []
  for i in text:
    if i not in stopwords:
      nonstopwords.append(i)
  return nonstopwords
rev['tokenized_text'] = rev['tokenized_text'].apply(filter_stopwords)
rev['tokenized_text'] = rev['tokenized_text'].str.join(' ')
rev.head(2)

,Unnamed: 0,review_id,business_id,reviews_stars,date,text,tokenized_text
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!",Best thai food in Mesa and you get so much food for your money We have tried almost every thai place in area and think Papaya thai is by far best and always ends p being cheapest too Only eating here for thair food for now
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The...",There are several contributing factors to starness of Papaya Thai but isn't necessarily food This place is literally a block away from my house and have ventured frequently because of proximity reasonable pricing and portioning Giant fish tanks with some cool tropical fish are strewn about but then there is tank with ugliest eel like fish and catfish It makes hard for me to eat looking at them so opt for seats facing opposite direction also feel like there are too many tables for amount of space If restaurant where ever full think would knock into someone's head with imaginary purse that do not carry Now onto food love that there are spiciness scales that do not adhere too always ask extra extra spicy and just throw a handful of red chile flakes into dish that order Every curry dish have ordered has tasted way should not like coconut milk or peanuts The price isn't terrible for portion always have enough left over for lunch next day Try out P S think are under new management since ...


In [14]:
rev_sample = rev[:1000]

In [15]:
rev_sample['positive_words'] = rev_sample['tokenized_text'].apply(positive)
rev_sample['negative_words'] = rev_sample['tokenized_text'].apply(negative)
rev_sample['positive_words'] = rev_sample['positive_words'].str.join(' ')
rev_sample['negative_words'] = rev_sample['negative_words'].str.join(' ')
rev_sample

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,Unnamed: 0,review_id,business_id,reviews_stars,date,text,tokenized_text,positive_words,negative_words
0,0,Oa-sapLub0wRAzxRKVmtDQ,j9bWpCRwpDVfwVT_V85qeA,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!",Best thai food in Mesa and you get so much food for your money We have tried almost every thai place in area and think Papaya thai is by far best and always ends p being cheapest too Only eating here for thair food for now,best best cheapest,
1,1,NawvdCkVIvhMO_eMVMdBhw,j9bWpCRwpDVfwVT_V85qeA,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The...",There are several contributing factors to starness of Papaya Thai but isn't necessarily food This place is literally a block away from my house and have ventured frequently because of proximity reasonable pricing and portioning Giant fish tanks with some cool tropical fish are strewn about but then there is tank with ugliest eel like fish and catfish It makes hard for me to eat looking at them so opt for seats facing opposite direction also feel like there are too many tables for amount of space If restaurant where ever full think would knock into someone's head with imaginary purse that do not carry Now onto food love that there are spiciness scales that do not adhere too always ask extra extra spicy and just throw a handful of red chile flakes into dish that order Every curry dish have ordered has tasted way should not like coconut milk or peanuts The price isn't terrible for portion always have enough left over for lunch next day Try out P S think are under new management since ...,reasonable cool like like love like enough,tanks tank ugliest hard knock imaginary terrible
2,2,uBWpaM4_PQ1NMB3N9nYkJA,j9bWpCRwpDVfwVT_V85qeA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price...",Moving to Phoenix area was a shock to begin with and has been difficult finding truly great Thai

In [61]:
positive_list = rev_sample['positive_words'].str.split(expand=True).stack().value_counts().rename_axis('words').reset_index(name='counts')
positive_list.head(10)

,words,counts
0,good,468
1,like,360
2,great,308
3,love,184
4,hot,164
5,cute,155
6,delicious,139
7,best,116
8,sweet,114
9,well,108


In [62]:
top_10_pos = positive_list.head(10)

In [63]:
negative_list = rev_sample['negative_words'].str.split(expand=True).stack().value_counts().rename_axis('words').reset_index(name='counts')
negative_list.head(10)

,words,counts
0,fudge,105
1,split,99
2,bad,98
3,dirty,67
4,fried,65
5,hard,56
6,disappointed,53
7,frozen,47
8,shake,45
9,cold,41


In [64]:
top_10_neg = negative_list.head(10)

In [65]:
top_10_pos = top_10_pos.to_json() 
print(top_10_pos) 

{"words":{"0":"good","1":"like","2":"great","3":"love","4":"hot","5":"cute","6":"delicious","7":"best","8":"sweet","9":"well"},"counts":{"0":468,"1":360,"2":308,"3":184,"4":164,"5":155,"6":139,"7":116,"8":114,"9":108}}


In [66]:
top_10_neg = top_10_neg.to_json() 
print(top_10_neg) 

{"words":{"0":"fudge","1":"split","2":"bad","3":"dirty","4":"fried","5":"hard","6":"disappointed","7":"frozen","8":"shake","9":"cold"},"counts":{"0":105,"1":99,"2":98,"3":67,"4":65,"5":56,"6":53,"7":47,"8":45,"9":41}}
